#### Code to download PV data.

Updated: March 25, 2025

lol??

I was not sure what to use as the isentropic level so I opted for 350, which I saw on brief glance roughl equated to the tropopause. 

In [1]:
#import statements... I think these are all of the relevant ones to what I am doing here. 
import datetime as dt
import numpy as np
import xarray as xr
import math
import netCDF4
import os
import scipy.stats
import pickle
import pandas as pd

In [2]:
##definition statement for unpacking files
def pvfile(year, month):
    file = xr.open_dataset("/network/rit/lab/wulab/Datasets/ERA5/6hr/ptlev/pv/era5_an_pvonpt_reg2_6h_"+year+month+".nc")
    daily = file.resample(time='1D').mean() ##convert to 0Z only
    daily['time'] = daily['time'].dt.date ##remove time from data
    pvfile = daily.loc[dict(lev = 350, lat=slice(90,60))] ## return xarray of desired data, PV over cap at 10hPa 

    return pvfile

In [3]:
##establish month list
months = ['10', '11', '12', '01', '02', '03']

In [11]:
#create empty arrays for the years of data
pv = np.empty((63,183,16,180))
pv[:] = np.nan

In [12]:
#loop years (1959 to 2021)
for year_index in range(63):
    year = str(1959 + year_index)
    print("start:",year)
    #loop months
    for month_index, month in enumerate(months):
        #day range based on month
        if month == '10':  # 10/1 to 10/31
            day_range = range(1, 32)
            day_index = range(0, 31)

            #upload appropriate pv data
            pv_data = pvfile(year, month)
            
        elif month == '11':  #11/1 to 11/30
            day_range = range(1, 31)
            day_index = range(31, 61)
            #upload appropriate pv data
            pv_data = pvfile(year, month)
            
        elif month == '12':  #12/1 to 12/31
            day_range = range(1, 32)
            day_index = range(61, 92)
            #upload appropriate pv data
            pv_data = pvfile(year, month)
            
        elif month == '01':  #1/1 to 1/31
            year = str(int(year)+1)
            print("add year:",year)
            day_range = range(1, 32)
            day_index = range(92, 123)
            #upload appropriate pv data
            pv_data = pvfile(year, month)
            
        elif month == '02':  # February 1st to February 28th (29th on leap years)
            print("check february:",year)
            if year == str(1960) or year == str(1964) or year == str(1968) or year == str(1972) or year == str(1976) or year == str(1980) or year == str(1984) or year == str(1988) or year == str(1992) or year == str(1996) or year == str(2000) or year == str(2004) or year == str(2008) or year == str(2012) or year == str(2016) or year == str(2020):
                day_range = range(1, 29)
                day_index = range(123, 152)
                #upload appropriate pv data
                pv_data = pvfile(year, month)
                
            else:
                day_range = range(1, 28)
                day_index = range(123, 151)
                #upload appropriate pv data
                pv_data = pvfile(year, month)
                
        elif month == '03':  # March 1st to March 31st
            day_range = range(1, 32)
            day_index = range(152, 183)
            #upload appropriate pv data
            pv_data = pvfile(year, month)
        
        #loop through days and save the data
        for day in day_range:
            date = dt.date(int(year), int(month), day)
            # Adjust the day index and save the data
            pv[year_index, day_index[day-1], :, :] = pv_data.sel(time=date)['var60'].values
    
    print("##################")


start: 1959
add year: 1960
check february: 1960
##################
start: 1960
add year: 1961
check february: 1961
##################
start: 1961
add year: 1962
check february: 1962
##################
start: 1962
add year: 1963
check february: 1963
##################
start: 1963
add year: 1964
check february: 1964
##################
start: 1964
add year: 1965
check february: 1965
##################
start: 1965
add year: 1966
check february: 1966
##################
start: 1966
add year: 1967
check february: 1967
##################
start: 1967
add year: 1968
check february: 1968
##################
start: 1968
add year: 1969
check february: 1969
##################
start: 1969
add year: 1970
check february: 1970
##################
start: 1970
add year: 1971
check february: 1971
##################
start: 1971
add year: 1972
check february: 1972
##################
start: 1972
add year: 1973
check february: 1973
##################
start: 1973
add year: 1974
check february: 1974
##############

In [13]:
pv[0,160:,0,0]

array([8.41446763e-06, 8.69925680e-06, 6.96183679e-06, 7.53383483e-06,
       7.54275516e-06, 7.41707026e-06, 8.00194448e-06, 8.17525142e-06,
       8.38091182e-06, 8.02983232e-06, 8.46899820e-06, 8.38083724e-06,
       8.23416849e-06, 7.76096022e-06, 8.00188900e-06, 8.58689054e-06,
       9.07270714e-06, 8.45651448e-06, 8.00242560e-06, 8.96209076e-06,
       7.89125352e-06, 8.18400258e-06, 8.73544013e-06])

In [14]:
##need to multiply the array by 10**6 to get in PVU
pvu = pv[:,:,:,:]*(1*10**6)

In [15]:
print(pvu[:,30:,:,:].shape)
#print(pvu[20,:,0,0])

(63, 153, 16, 180)


In [16]:
pickle.dump(pvu[:,:,:,:], open("cap_pv350pt.p", 'wb'))